
# Notebook ReAct aplicada

Clase aplicada de ReAct con herramientas simples, trazabilidad y feedback loop de calidad.


In [1]:

from __future__ import annotations

import json
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

ROOT = Path.cwd()
sys.path.append(str((ROOT / "02-prompting").resolve()))
from common.rubrica import evaluar_salida

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Falta OPENAI_API_KEY en .env")

model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
client = OpenAI(api_key=api_key)


In [2]:

perfil = {
    "tipo_persona": "consultora de marca personal",
    "gustos": ["arte contemporaneo", "vino", "estrategia de negocio"],
    "estilo": "calida, analitica y directa",
    "contexto": "conversacion retomada despues de varios dias",
}

def analizar_perfil(p):
    return {
        "insights": [
            f"Intereses clave: {', '.join(p.get('gustos', []))}",
            "Conviene una apertura breve con pregunta concreta.",
        ],
        "tono": p.get("estilo", "calido"),
    }

def auditar_respeto(salida):
    texto = f"{salida.get('opener', '')} {salida.get('follow_up', '')}".lower()
    flags = [x for x in ["presion", "insistir", "explicito"] if x in texto]
    return {"ok": len(flags) == 0, "flags": flags}

analysis = analizar_perfil(perfil)
analysis


{'insights': ['Intereses clave: arte contemporaneo, vino, estrategia de negocio',
  'Conviene una apertura breve con pregunta concreta.'],
 'tono': 'calida, analitica y directa'}

In [3]:

prompt_react = f"""
Usa formato ReAct con esta secuencia:
Thought -> Action -> Observation -> Thought -> Action -> Observation -> Final Answer

Herramientas disponibles:
- analizar_perfil: {json.dumps(analysis, ensure_ascii=False)}
- auditar_respeto: se ejecuta despues de proponer mensaje

Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Genera JSON:
{{
  "trace": [
    {{"thought": "...", "action": "...", "observation": "..."}},
    {{"thought": "...", "action": "...", "observation": "..."}}
  ],
  "result": {{
    "opener": "...",
    "follow_up": "...",
    "why_it_works": ["...", "..."]
  }}
}}

No uses emojis.
"""

resp = client.chat.completions.create(
    model=model,
    temperature=0.5,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres un agente ReAct con enfoque profesional y respetuoso."},
        {"role": "user", "content": prompt_react},
    ],
)

react_out = json.loads(resp.choices[0].message.content)
audit = auditar_respeto(react_out["result"])
eval_react = evaluar_salida(perfil, react_out["result"])

print(json.dumps(react_out, ensure_ascii=False, indent=2))
print(json.dumps(audit, ensure_ascii=False, indent=2))
print(json.dumps(eval_react, ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "Es importante retomar la conversación de manera cálida y directa, dado el tiempo que ha pasado desde el último contacto.",
      "action": "Proponer una apertura breve que invite a la otra persona a participar.",
      "observation": "Una pregunta concreta puede facilitar la reanudación de la conversación y mostrar interés en su perspectiva."
    },
    {
      "thought": "Dado su interés en arte contemporáneo, vino y estrategia de negocio, puedo conectar alguno de estos temas en mi seguimiento.",
      "action": "Plantear una pregunta que relacione sus intereses con la conversación actual.",
      "observation": "Esto puede generar un diálogo más enriquecedor y alineado con sus gustos."
    }
  ],
  "result": {
    "opener": "¿Cómo has estado? Me encantaría saber tu opinión sobre cómo el arte contemporáneo puede influir en las estrategias de negocio actuales.",
    "follow_up": "Además, ¿has tenido la oportunidad de disfrutar de algún buen vino ú

In [4]:

prompt_refine = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Salida actual:
{json.dumps(react_out, ensure_ascii=False, indent=2)}

Auditoria:
{json.dumps(audit, ensure_ascii=False, indent=2)}

Rubrica:
{json.dumps(eval_react, ensure_ascii=False, indent=2)}

Mejora result.opener y result.follow_up con el mismo estilo ReAct.
No uses emojis.
Devuelve JSON con trace y result.
"""

resp2 = client.chat.completions.create(
    model=model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres auditor y optimizador de agentes ReAct."},
        {"role": "user", "content": prompt_refine},
    ],
)

mejorada = json.loads(resp2.choices[0].message.content)
print(json.dumps(mejorada, ensure_ascii=False, indent=2))
print(json.dumps(evaluar_salida(perfil, mejorada["result"]), ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "Es fundamental retomar la conversación de manera cálida y directa, considerando el tiempo que ha pasado desde el último contacto.",
      "action": "Proponer una apertura que no solo invite a participar, sino que también refleje sus intereses.",
      "observation": "Incluir referencias más claras a sus gustos puede hacer que la conversación sea más atractiva."
    },
    {
      "thought": "Al incorporar sus intereses en arte contemporáneo y vino, puedo hacer que la conversación sea más relevante y personal.",
      "action": "Formular una pregunta que conecte sus pasiones con la situación actual.",
      "observation": "Esto puede facilitar un diálogo más enriquecedor y alineado con sus gustos."
    }
  ],
  "result": {
    "opener": "¿Cómo has estado? Me gustaría saber tu perspectiva sobre cómo el arte contemporáneo está influyendo en las estrategias de negocio en este momento.",
    "follow_up": "Por cierto, ¿has descubierto algún vino interes